# Bottle Data

### Purpose
This notebook provides a quick outline and example of working with OOI Discrete Summary Spreadsheet bottle data. It includes how to load it, how to parse data quality flags, how to derive some important values, and work with the data to make it easy to use for data validation and comparison purposes.

### Background
The Ocean Observatories Initiative (OOI) is an NSF-funded project for long-term (25 years) infrastructure for ocean observations. There are three main elements of OOI: (1) The Regional Cabled Array off the coast of Washington; (2) the Endurance Array on the Oregon-Washington shelf; and (3) the Coastal & Global Scale Nodes with mooring arrays in the Irminger Sea (Irminger Sea Array), at Ocean Station Papa (Papa Array), in the Southern Ocean and Argentine Basin (descoped and no longer collecting data), and at the New England Shelf (Pioneer Array-NES).

OOI deploys a variety of instrumentation that aim to measure the physical, chemical, and biological components of ocean activity. For data validation and comparison purposes with instrumentation, OOI also performs CTD casts and collects Niskin bottles for laboratory analysis during deployment and recovery of each mooring at each instrument depth. The laboratory analysis include:
* Oxygen
* Salinity
* Nutrients (Nitrate, Nitrite, Ammonium, Phosphate, Silicate)
* Chlorophyll
* Carbon System (at least two of DIC/TA/pH/pCO2)

### Data
The cruise data is hosted on OOI's Alfresco document server at https://alfresco.oceanobservatories.org, hereafter reffered to as Alfresco. Cruise data is hosted under OOI > {Array} > Cruise Data > {Cruise}. Cruise data includes the following key pieces of information: Cruise Reports; Ship Data (e.g. sonars, underway instruments, etc.); CTD; and Water Sampling Data.

We are principally concerned with the Water Sampling data, which can be found under {Cruise} > Ship Data > Water Sampling. The Water Sampling folder contains both digitized and scanned CTD logs as well as the results from the labs for each measured parameter. Helpfully, the CTD Bottle Data (.btl files) have been merged with the laboratory results and the CTD Sampling Logs to create **Discrete Summary Spreadsheet** of the water sampling data. These are the best place to start when working with OOI Bottle Data and are the data sources that we will be using in this example notebook.

In [ ]:
import os, sys, re
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
sys.path.append("../../Tutorial/src")

In [ ]:
from pyOOI import Bottles

---
### Niskin & Discrete Bottle Sampling
These are discrete water samples collected via Niskin Bottle casts during deployment and recovery of the moored instrumentation. The data is downloaded from OOI Alfresco website as excel files. Parameters sampled are:
* Oxygen
* Salinity
* Nutrients: phosphate, nitrate, nitrite, ammonium, silicate
* Chlorophyll
* Carbon system: Dissolved inorganic carbon, total alkalinity, pH

First, we're going to load, clean, and process the discrete water sampling from Ocean Station Papa Array.

In [ ]:
sorted(os.listdir("../data/OOI/Ship_Data/"))

In [ ]:
cruise_file = "Station_Papa-05_SR1710_Discrete_Summary.xlsx"
bottleData = pd.read_excel("../data/OOI/Ship_Data/" + cruise_file)

Check on the available cruises

In [ ]:
bottleData

In [ ]:
bottleData["Cruise"].unique()

Replace the fill value of ```-9999999``` with NaNs. Some columns were imported as floats and others as strings or objects, so this needs to be done twice:

In [ ]:
bottleData = bottleData.replace(to_replace="-9999999", value=np.nan)
bottleData = bottleData.replace(to_replace=-9999999, value=np.nan)

Convert times from strings to pandas datetime objects and remove the timezone:

In [ ]:
bottleData["Start Time [UTC]"] = bottleData["Start Time [UTC]"].apply(lambda x: convert_times(x))
bottleData["CTD Bottle Closure Time [UTC]"] = bottleData["CTD Bottle Closure Time [UTC]"].apply(lambda x: convert_times(x))

Some of the data returned from the labs which process samples contain non-numeric indicators for if data were below detection threshold. These entries typically have a "less than" sign. Convert any values with a "<", which indicates a value not statistically significant from zero, with zero:

In [ ]:
bottleData = bottleData.applymap(not_statistically_sigificant)

#### Filter data
Next, we want to filter the data based on the data flag associated with the sample. First, we should drop all of the ```NaN``` values in the Niskin column to eliminate entries without any bottle samples. While these CTD casts may be of interest, the bottle data does not have any data of which we can make use of.

In [ ]:
bottleData = bottleData.dropna(subset=["Niskin/Bottle Position"])

#### Interpret Flags

Next, we want to parse the Data Quality Flags in the bottle data. We intrept the data into the standard QARTOD flaging scheme of:
* 1 = good
* 2 = not run
* 3 = suspect
* 4 = bad
* 9 = missing

The "Replicate Flags" are reduced into a boolean value indicating that either there is a replicate sample or not. By replicate, we mean if there are more than one sample collected from a depth, e.g. two Niskins were closed at a particular depth and oxygen sampled from both. It does *not* mean that a particular analysis was repeated on the same sample multiple times, e.g. repeating nutrient analysis on a single Niskin to derive a final value.

In [ ]:
for col in bottleData.columns:
    if "Flag" in col:
        if "CTD" in col and "File" not in col:
            bottleData[col] = bottleData[col].apply(lambda x: interp_ctd_flag(x))
        elif "Discrete" in col:
            bottleData[col] = bottleData[col].apply(lambda x: interp_discrete_flag(x))
        elif "Replicate" in col:
            bottleData[col] = bottleData[col].apply(lambda x: interp_replicate_flag(x))
        elif "Niskin" in col:
            bottleData[col] = bottleData[col].apply(lambda x: interp_niskin_flag(x))
        else:
            pass

---
### Calculate Physical Properties
With the bottle samples cleaned up and the flags parsed into something simpler and easier to read, we next want to calculate the conservative seawater properties, such as density, conservative temperature, absolute salinity, etc., using the updated Thermodynamic Equations of State - 2010 (TEOS-10) algorithms. These are implemented with the ```gsw``` package. 

This step will allow us to correct for pressure and temperature effect between lab analysis and in-situ measurements, as well as allow for better comparison between depths and locations in the same array. 

In [ ]:
import gsw

Get the measured parameters, taking the average if measured by more than one sensor

In [ ]:
SP = bottleData[["CTD Salinity 1 [psu]", "CTD Salinity 2 [psu]"]].mean(axis=1, skipna=True)
T = bottleData[["CTD Temperature 1 [deg C]", "CTD Temperature 2 [deg C]"]].mean(axis=1, skipna=True)
P = bottleData["CTD Pressure [db]"]
LAT = bottleData["CTD Latitude [deg]"]
LON = bottleData["CTD Longitude [deg]"]

Calculate the conservative temperature, absolute salinity, density, and potential density of the observations

In [ ]:
# Absolute salinity
SA = gsw.conversions.SA_from_SP(SP, P, LON, LAT)
bottleData["CTD Absolute Salinity [g/kg]"] = SA

# Conservative temperature
CT = gsw.conversions.CT_from_t(SA, T, P)
bottleData["CTD Conservative Temperature"] = CT

# Density
RHO = gsw.density.rho(SA, CT, P)
bottleData["CTD Density [kg/m^3]"] = RHO

# Calculate potential density
SIGMA0 = gsw.density.sigma0(SA, CT)
bottleData["CTD Sigma [kg/m^3]"] = RHO

del SP, T, P, LAT, LON, SA, CT, RHO, SIGMA0

---
## Calculate Carbon System Parameters
The discrete water samples were tested for Total Alkalinity, Dissolved Inorganic Carbon, and pH [Total Scale]. We can calculate the discrete water sample pCO<sub>2</sub> concentrations from the TA and DIC using the ```CO2SYS``` program. Implementation in python is achieved using the PyCO2SYS package. Full documentation on how to install and use PyCO2SYS can be found at https://pyco2sys.readthedocs.io/en/latest/. 

For our purposes, we will be utilizing the following inputs:
* ```PAR1```: First measured carbon system measurement
* ```PAR2```: Second measured carbon system measurement
* ```PAR1_type```: The type of PAR1 
        * 1 = Total Alkalinity umol/kg
        * 2 = DIC umol/kg
        * 3 = pH Total Scale
        * 4 = pCO2
        * 5 = fCO2
* ```PAR2_type```: The type of PAR2


We will also use the following optional hydrographic inputs:
* ```salinity```: practical salinity
* ```temperature```: the temperature at which PAR1 and PAR2 are provided (in C)
* ```pressure```: the water pressure at which ```PAR1``` and ```PAR2``` are measured

Additionally, we will pass in some of the nutrient data that we collect:
* ```SIO4```: discrete silicate
* ```PO4```: discrete phosphate
* ```NH4```: discrete ammonia

The nutrient values need to be (1) converted to umol/kg from uM and (2) have NaNs filled in with 0, which is the default value.

In [ ]:
import PyCO2SYS as pyco2

In [ ]:
# Get the key parameters
DIC = bottleData["Discrete DIC [umol/kg]"]
TA = bottleData["Discrete Alkalinity [umol/kg]"]
PH = bottleData["Discrete pH [Total scale]"]
#SAL = bottleData["Discrete Salinity [psu]"]
SAL = bottleData[["CTD Salinity 1 [psu]", "CTD Salinity 2 [psu]"]].mean(axis=1, skipna=True)

# Set the input hydrographic parameters at which the DIC/TA/pH lab measurements were performed
# TEMP_IN = bottleData["pH Analysis Temp [deg C]"].replace(to_replace=np.nan, value=25)
TEMP_IN = bottleData[["CTD Temperature 1 [deg C]", "CTD Temperature 2 [deg C]"]].mean(axis=1, skipna=True)
PRES_IN = bottleData["CTD Pressure [db]"]

# Get the hydrographic parameters at which the samples were taken
TEMP_OUT = bottleData[["CTD Temperature 1 [deg C]","CTD Temperature 2 [deg C]"]].mean(axis=1, skipna=True)
PRES_OUT = bottleData["CTD Pressure [db]"]
RHO = bottleData["CTD Density [kg/m^3]"]

# Nutrient inputs = need to fill NaNs with zeros otherwise will return NaNs
SIO4 = bottleData["Discrete Silicate [uM]"].astype(float).fillna(value=0)
PO4 = bottleData["Discrete Phosphate [uM]"].fillna(value=0)
NH4 = bottleData["Discrete Ammonium [uM]"].fillna(value=0)

# Convert to umol/kg
SIO4 = SIO4*(1000/1)*(1/RHO).fillna(value=0)
PO4 = PO4*(1000/1)*(1/RHO).fillna(value=0)
NH4 = NH4*(1000/1)*(1/RHO).fillna(value=0)

In [ ]:
TEMP_IN

Declare the parameter input types to pass into ```CO2SYS```. This is going to vary for Ocean Station Papa based on who did the cruise and what lab processed the bottles:

In [ ]:
cruise_file

In [ ]:
#PAR1 = bottleData['Discrete Alkalinity [umol/kg]']
PAR1 = bottleData["Discrete pCO2 [uatm]"]
PAR2 = bottleData['Discrete DIC [umol/kg]']
#PAR1TYPE = 1
PAR1TYPE = 4
PAR2TYPE = 2
PHSCALEIN = 1
K1K2CONSTANTS = 1
K2SO4CONSTANTS = 1

#### Calculate Carbon System
With all of the mandatory and optional carbon system parameters selected, we now calculate the carbon system for the bottle data

In [ ]:
CO2dict = pyco2.CO2SYS(PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMP_IN, TEMP_OUT, PRES_IN, PRES_OUT, SIO4, PO4, PHSCALEIN, K1K2CONSTANTS, K2SO4CONSTANTS)

Now add the calculate carbon system parameters to the bottle data

In [ ]:
bottleData['Calculated Alkalinity [umol/kg]'] = CO2dict['TAlk']
bottleData['Calculated CO2aq [umol/kg]'] = CO2dict['CO2out']
bottleData['Calculated CO3 [umol/kg]'] = CO2dict['CO3out']
bottleData['Calculated DIC [umol/kg]'] = CO2dict['TCO2']
bottleData['Calculated pCO2 [uatm]'] = CO2dict['pCO2out']
#bottleData['Calculated pCO2 [lab conditions]'] = CO2dict['pCO2in']
#bottleData['Calculated pH [lab conditions]'] = CO2dict['pHoutTOTAL']
derivedPH = pd.Series(data=CO2dict["pHoutTOTAL"], index=bottleData.index)
bottleData['Calculated pH'] = CO2dict["pHoutTOTAL"]

In [ ]:
CO2dict["pHoutTOTAL"]

#### Adjust Discrete pH

Next, we need to adjust the **Discrete pH** to in-situ conditions. We'll do this by passing in the measured **Discrete pH** along with the **Discrete Alkalinity** and adjusting it to the in-situ conditions. To do this we only need to adjust the previous parameters such that ```PAR2``` is the **Discrete pH \[Total Scale\]** and ```PAR2TYPE``` is set to **3**.

In [ ]:
#PAR1 = bottleData['Discrete Alkalinity [umol/kg]']
PAR1 = bottleData["Discrete DIC [umol/kg]"]
#PAR2 = bottleData['Discrete pH [Total scale]']
PAR2 = bottleData["Calculated pH"]
#PAR1TYPE = 1
PAR1TYPE = 2
PAR2TYPE = 3
PRES_OUT = 0
TEMP_IN = TEMP_OUT
#TEMP_OUT = bottleData["pCO2 Analysis Temp [deg C]"].fillna(25)
TEMP_OUT = 25

In [ ]:
PRES_IN = bottleData["CTD Pressure [db]"]


Recalculate the carbon system to get the in-situ pH

In [ ]:
CO2dict = pyco2.CO2SYS(PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMP_IN, TEMP_OUT, PRES_IN, PRES_OUT, SIO4, PO4, PHSCALEIN, K1K2CONSTANTS, K2SO4CONSTANTS)

In [ ]:
#bottleData["Calculated pH"] = CO2dict["pHoutTOTAL"]
bottleData["Calculated pH [lab conditions]"] = CO2dict["pHoutTOTAL"]
# Fill in missing "Calculate pH" with the derived pH to increase the number of pH observations
#bottleData["Calculated pH"].fillna(value=derivedPH, inplace=True)
bottleData[["Discrete pH [Total scale]","Calculated pH [lab conditions]","Calculated pH"]].dropna(subset=["Calculated pH"])

---
### Save the Results
Finally, with the carbon system results fully derived, save the results to the data directory with the other bottle data (or wherever you want to save the data)

In [ ]:
cruise_file
save_file = cruise_file.split(".")[0] + "_pH"
save_file

In [ ]:
bottleData.to_csv(f"../data/OOI/Ship_Data/{save_file}.csv", index=False)